# Data Concatenator

## 필요한 Data
- surface level ERA5, precipitation 제외
- surface level ERA5, precipitation 포함
- pressure level ERA5
- (optional) TOA solar incident radiation

cmd에서 `.nc`로 변환하는 법
```
module load gcc/cdo-1.9.3
cdo -f nc copy input.grib output.nc
```

In [12]:
import xarray as xr
import numpy as np
import his_utils
import dask

In [13]:
# set up dataset path 
is_grib = False

if is_grib:
    ds_surface = "testdata/2021-06-26/adaptor.mars.internal-1724031931.2125857-25557-2-0d36c803-bee9-46ae-b776-90ad84607ec1.grib"
    ds_surface_precip = "testdata/2021-06-26/adaptor.mars.internal-1724030587.1234627-16629-11-0d73df6e-8237-4e6e-ba02-ebb18a25bea3.grib"
    ds_pressure_level = "testdata/2021-06-26/adaptor.mars.internal-1724030615.4343812-27325-8-43695fb3-3e45-4d8e-9868-02450d88f8e2.grib"
    ds_surface_precip_prior = None
else:
    ds_surface = "testdata/2021-06-26/surface.nc"
    ds_surface_precip = "testdata/2021-06-26/precip.nc"
    ds_surface_precip_prior = None
    ds_pressure_level = "testdata/2021-06-26/pressure.nc"
# ds_TOA = "testdata/2022-01-01/2022-01-01 TOA.grib"



# 1. Surface Data w/o Precipitation

precipitation 제외하고 받은 6-hrly data 전처리 하는 과정

## TODO
- dimension 이름 변경
- 불필요한 coordinate 삭제
- variable 이름 변경 -> 나중에 합치고 한 번에 진행 가능

In [14]:
if is_grib:
    ds1 = xr.open_dataset(ds_surface, engine='cfgrib')
    ds1 = ds1.drop_vars(['number', 'step', 'surface', 'valid_time'])
    ds1 = ds1.rename({"z" : "geopotential_at_surface"})

else:
    ds1 = xr.open_dataset(ds_surface)
    ds1 = ds1.rename({'var165':"u10",
                      'var166':"v10", 
                      'var167':"t2m", 
                      'var129':"geopotential_at_surface", 
                      'var172':"lsm", 
                      'var151':"msl", 
                      'var212':"tisr"})
ds1

skipping variable: paramId==212 shortName='tisr'
Traceback (most recent call last):
  File "/home/hiskim1/.conda/envs/hiskim1_graphcast/lib/python3.11/site-packages/cfgrib/dataset.py", line 721, in build_dataset_components
    dict_merge(variables, coord_vars)
  File "/home/hiskim1/.conda/envs/hiskim1_graphcast/lib/python3.11/site-packages/cfgrib/dataset.py", line 639, in dict_merge
    raise DatasetBuildError(
cfgrib.dataset.DatasetBuildError: key present and new value is different: key='time' value=Variable(dimensions=('time',), data=array([1624665600, 1624687200, 1624708800, 1624730400, 1624752000,
       1624773600, 1624795200, 1624816800, 1624838400, 1624860000,
       1624881600, 1624903200, 1624924800, 1624946400, 1624968000,
       1624989600, 1625011200, 1625032800, 1625054400, 1625076000])) new_value=Variable(dimensions=('time',), data=array([1624644000, 1624687200, 1624730400, 1624773600, 1624816800,
       1624860000, 1624903200, 1624946400, 1624989600, 1625032800]))


<xarray.Dataset>
Dimensions:                  (time: 20, latitude: 721, longitude: 1440)
Coordinates:
  * time                     (time) datetime64[ns] 2021-06-26 ... 2021-06-30T...
  * latitude                 (latitude) float64 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude                (longitude) float64 0.0 0.25 0.5 ... 359.5 359.8
Data variables:
    u10                      (time, latitude, longitude) float32 ...
    v10                      (time, latitude, longitude) float32 ...
    t2m                      (time, latitude, longitude) float32 ...
    geopotential_at_surface  (time, latitude, longitude) float32 ...
    lsm                      (time, latitude, longitude) float32 ...
    msl                      (time, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-08-19T13:59 GRIB to CDM+CF via cfgrib-0.9.1...

# 2. Precipitation Data

Accumulates 6-hour precipitation into the next time step?
or into previous step?

## TODO
- coordinate 정리
- 강수량 합치기

In [15]:
# 2-1. the time after
def sync_tp_coords(dataset: xr.Dataset):
    dataset = dataset.stack(new_time=['time', 'step'])
    dataset = dataset.assign_coords(new_time=dataset.valid_time.values)
    dataset = dataset.rename({'new_time': 'time'})
    dataset = dataset.drop_vars(['number', 'surface'])
    return dataset

if is_grib:
    ds2 = xr.open_dataset(ds_surface_precip, engine='cfgrib')
    if ds_surface_precip_prior is not None:
        prior = xr.open_dataset(ds_surface_precip_prior, engine='cfgrib')

    ds2 = sync_tp_coords(ds2)
    ds2 = ds2.isel(time=slice(5,None)) #                                          <------- FIX HERE
else:
    ds2 = xr.open_dataset(ds_surface_precip)
    ds2 = ds2.rename({'var228': 'tp'})
    if ds_surface_precip_prior is not None:
        prior = xr.open_dataset(ds_surface_precip_prior)
        prior = prior.rename({'var228': 'tp'})
ds2

/home/hiskim1/.conda/envs/hiskim1_graphcast/lib/python3.11/site-packages/xarray/core/common.py:615: FutureWarning: Updating MultiIndexed coordinate 'new_time' would corrupt indices for other variables: ['time', 'step']. This will raise an error in the future. Use `.drop_vars({'new_time', 'step', 'time'})` before assigning new coordinate values.
  data.coords.update(results)


<xarray.Dataset>
Dimensions:     (latitude: 721, longitude: 1440, time: 151)
Coordinates:
  * latitude    (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
    valid_time  (time) datetime64[ns] 2021-06-25 ... 2021-07-01T06:00:00
  * time        (time) datetime64[ns] 2021-06-25 ... 2021-07-01T06:00:00
Data variables:
    tp          (latitude, longitude, time) float32 4.768e-07 9.537e-07 ... nan
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-08-19T13:59 GRIB to CDM+CF via cfgrib-0.9.1...

In [16]:
# 2-2. the time before
if is_grib and ds_surface_precip_prior is not None:
    prior = sync_tp_coords(prior)
    prior = prior.isel(time=slice(5,29)) #                                        <------- FIX HERE
else:
    pass

In [17]:
# 2-3. merge the two datasets
if ds_surface_precip_prior is not None:
    ds2 = xr.concat([prior, ds2], dim='time')
else:
    pass

ds2 = ds2.sortby('time')
ds2 = ds2.resample(time='6h', closed='right', label='right').sum()
ds2 = ds2.isel(time=slice(4, 24)) #                                           <------- FIX HERE
ds2

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 20)
Coordinates:
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[ns] 2021-06-26 ... 2021-06-30T18:00:00
Data variables:
    tp         (time, latitude, longitude) float32 8.583e-06 ... 9.68e-05
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-08-19T13:59 GRIB to CDM+CF via cfgrib-0.9.1...

# 3. Pressure Level Data

37 level data를 처리하는 과정. 17 level이어도 동일한 방식

## TODO
- coordinate 정리

In [18]:
if is_grib:
    ds3 = xr.open_dataset(ds_pressure_level, engine='cfgrib')
    ds3 = ds3.drop_vars(['number', 'step', 'valid_time'])
    ds3 = ds3.rename({"isobaricInhPa" : "level"})
    ds3 = ds3.sortby('level', ascending=True)
else:
    ds3 = xr.open_dataset(ds_pressure_level)
    ds3 = ds3.rename({'var130':"t", 
                      'var131':"u", 
                      'var132':"v", 
                      'var129':"z", 
                      'var133':"q", 
                      'var135':"w",
                      'plev': "level"})
    ds3['level'] = ds3['level']/100
    ds3['level'].attrs['units'] = 'hPa'
    ds3['level'].attrs['long_name'] = 'pressure level'
ds3

<xarray.Dataset>
Dimensions:    (time: 20, level: 37, latitude: 721, longitude: 1440)
Coordinates:
  * time       (time) datetime64[ns] 2021-06-26 ... 2021-06-30T18:00:00
  * level      (level) float64 1.0 2.0 3.0 5.0 7.0 ... 925.0 950.0 975.0 1e+03
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
Data variables:
    z          (time, level, latitude, longitude) float32 ...
    q          (time, level, latitude, longitude) float32 ...
    t          (time, level, latitude, longitude) float32 ...
    u          (time, level, latitude, longitude) float32 ...
    v          (time, level, latitude, longitude) float32 ...
    w          (time, level, latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-08-19T13:59 GRIB to CDM+CF via cfgrib-0.9.1...

In [19]:
level = ds3.level.values
level = level.astype(np.int32)

ds3 = ds3.assign_coords(level = ('level', level))

# (optional) 4. TOA 가공하기

구글에서 만든 거랑 내가 다운받은거랑 같다면 상관 없음.
다를 경우에는 이거 사용해야 함.

$\therefore$ $\exists$ noise
$\Rightarrow$ 결과가 미묘하게 달라지지만 유의미해보이지는 않다

In [20]:
# 4th. TOA
# ds4 = xr.open_dataset(ds_TOA, engine='cfgrib')

# ds4 = ds4.stack(new_time=['time', 'step'])
# ds4 = ds4.assign_coords(new_time=ds4.valid_time.values)
# ds4 = ds4.rename({'new_time': 'time'})
# ds4 = ds4.drop_vars(['number', 'surface', 'valid_time'])

# 4. 3개의 데이터셋을 하나로 합성하기

*ds1*, *ds2*, *ds3*, (*ds4*)를 하나로 합치고 GC에 잘 들어가도록 다듬어주기

## TODO
- 한 장씩 합치는 게 좋을지, 여러 장 한 번에 합치는 게 좋을지 for memory efficiency
- 합쳐서 GC에 잘 들어가는 지까지 확인

In [21]:
# merge all datasets
# ds_list = [ds1, ds2, ds3, ds4]
ds_list = [ds1, ds2, ds3]

result = xr.merge(ds_list)

for ds in ds_list:
    ds.close()

result = his_utils.transform_dataset(result)

## 4-1. 파일 명 정하기

In [22]:
result_path = 'testdata/2021-06-26/ERA5_input.nc'  #                     <------- FIX HERE

# 5. Output: GC로 준비 갈 완료!

In [23]:
result.to_netcdf(result_path)

---
---
---
---
---
---
---
---
---
---
---
---

# 번외: xarray.resample은 어떻게 작동하는가?

| 1 | 2 | 3 | 4 | 5 | 6 | 7 | value

| 0 | 1 | 2 | 3 | 4 | 5 | 6 | hr

`closed=` 어느 쪽을 닫힌 구간으로 쓸 것인가 $\Rightarrow$ 어느 쪽을 포함하고 반대쪽을 제외할까

`label=` sample한 거를 어느 쪽에 할당할 것인가

- case 1) `xarray.resample("6h")`

    1 + ... + 6을 0hr에 할당

- case 2) `xarray.resample("6h", closed='right', label='right')`

    2 + ... + 7을 6hr에 할당

In [ ]:
import pandas as pd

# 샘플 데이터 생성 (0시부터 23시까지)
date_range = pd.date_range(start='2021-12-31T19:00:00.000000000', end='2022-01-05T06:00:00.000000000', freq='h')
data = np.arange(1, 109, 1)
ds = pd.Series(data, index=date_range)

# 6시간 간격으로 리샘플링
ds_resampled = ds.resample('6h', closed='right', label='right').sum()

print("\n리샘플링 결과:")
print(ds_resampled)

# 각 리샘플링 구간의 시작과 끝 확인
for i, value in ds_resampled.items():
    start = i
    end = i + pd.Timedelta(hours=5)
    original_data = ds[start:end]
    print(f"\n{i}의 리샘플링 구간:")
    print(f"시작: {start}, 끝: {end}")
    print("포함된 원본 데이터:")
    print(original_data)
    print(f"합계: {value}")